Reference: I cooperated with Xinyu Wei on this homework

In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import time
import itertools
from itertools import combinations,permutations
import scipy

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/wangzh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
stop_words = set(stopwords.words('english'))

In [4]:
with open('../wiki-text.txt','r') as f:
    text = [line.split() for line in f]  

In [5]:
len(text[0])

124301826

# Data preprocessing

In [6]:
from collections import Counter
c_text = Counter(text[0])

In [7]:
min_threshold = 500
#text_filter1 = [k for k, v in c_text.items() if v > min_threshold]
start = time.time()
text_fil1= [word for word in text[0] if not word in stop_words]
text_filtered = [k for k in text_fil1 if c_text[k] > min_threshold]
end = time.time()
print("time elapsed: " + str(end-start))
print(len(set(text_filtered)))


time elapsed: 46.019356966
13201


In [8]:
len(text_filtered)

71618337

In [9]:
#text_filtered = text_filtered[:50000]
vocab = list(set(text_filtered))

In [42]:
len(vocab)

13201

# PMI Embedding

In [10]:
vocab_index = {}
for i,w in enumerate(vocab):
    vocab_index[w] = i

## First, create a word count matrix

In [44]:
from scipy.sparse import csr_matrix
from scipy.sparse import lil_matrix


In [45]:
def my_WW():
    start_0 = time.time()
    text_t = text_filtered
    N = len(text_t)
    window_size = 5
 
    WW = lil_matrix((len(vocab),len(vocab)), dtype = np.float64)
    
    for i, w in enumerate(text_t):
        if i % 1000000 == 0:
            end = time.time()
            percen = float(i)/len(text_t)
            print("WW processed completed: "+str(percen))
            print("time elapsed: "+ str(end - start_0))
        
        stepsize = min(window_size,N-i)
        window = text_t[i:i+stepsize+1]
        pairs = list(itertools.permutations(window,2))
        for p in pairs:
            WW[vocab_index[p[0]],vocab_index[p[1]]] += 1
    
    end_0 = time.time()
    print("WW stage complete!!!")
    print("time elapsed: "+ str(end_0-start_0))
    return(WW)





    
    
                

        

## Compute PMI matrix

In [46]:
start = time.time()
WW = my_WW()
end = time.time()
print("time elapsed: "+ str(end - start))
SP = WW.count_nonzero()
WW = WW.toarray()
Ni = WW.sum(axis = 1)
Nis = np.diagflat(1/Ni)
M = (WW+1)*SP
M = np.dot(Nis,M).dot(Nis)
M = np.log(M)
end = time.time()
print("time elapsed: "+ str(end - start))

WW processed completed: 0.0
time elapsed: 5.77206707001
WW processed completed: 0.0139629045003
time elapsed: 144.350976944
WW processed completed: 0.0279258090006
time elapsed: 285.439615011
WW processed completed: 0.0418887135008
time elapsed: 428.00762105
WW processed completed: 0.0558516180011
time elapsed: 571.069114923
WW processed completed: 0.0698145225014
time elapsed: 715.16218996
WW processed completed: 0.0837774270017
time elapsed: 858.874161959
WW processed completed: 0.097740331502
time elapsed: 1003.24388194
WW processed completed: 0.111703236002
time elapsed: 1146.890414
WW processed completed: 0.125666140503
time elapsed: 1290.74757385
WW processed completed: 0.139629045003
time elapsed: 1434.50325394
WW processed completed: 0.153591949503
time elapsed: 1578.55832887
WW processed completed: 0.167554854003
time elapsed: 1723.28129005
WW processed completed: 0.181517758504
time elapsed: 1867.77483106
WW processed completed: 0.195480663004
time elapsed: 2012.65559888
WW p

# (b) k-SVD

In [47]:
start = time.time()
U,s,V = scipy.sparse.linalg.svds(scipy.sparse.csr_matrix(M),k = 50)
end = time.time()
print("time elapsed: "+ str(end - start))

time elapsed: 70.536842823


# (c) Word Embedding Matrix

In [48]:
start = time.time()
ss = np.diag(np.sqrt(s))
W = np.dot(U,ss)
end = time.time()
print("time elapsed: "+ str(end - start))

time elapsed: 0.0049729347229


# (d) Find closed words

In [11]:
def closest_word(word,W,num=6):
    print("closest words to "+ word, "are: ")
    start = time.time()
    key = vocab_index[word]
    vec = W[key]
    diff_W = W - np.array([vec]*W.shape[0])
    diff = [np.dot(x,x) for x in diff_W]
    close = np.argpartition(diff,num)[:num]
    
    for i in close:
        if i!= key:
            print(vocab[i])
    end = time.time()
    print("time elapsed: "+ str(end-start))
    

In [50]:
closest_word("abuse",W,6)

('closest words to abuse', 'are: ')
victim
homosexuality
torture
rape
psychiatric
time elapsed: 0.0139501094818


In [51]:
closest_word("physics",W,6)

('closest words to physics', 'are: ')
quantum
chemistry
mechanics
mathematics
theoretical
time elapsed: 0.0124969482422


In [52]:
closest_word("republican",W,6)

('closest words to republican', 'are: ')
senator
democrat
democrats
republicans
presidential
time elapsed: 0.0121231079102


In [53]:
closest_word("einstein",W,6)

('closest words to einstein', 'are: ')
planck
physicists
relativity
paradox
leibniz
time elapsed: 0.0128509998322


In [54]:
closest_word("algebra",W,6)

('closest words to algebra', 'are: ')
algebraic
calculus
finite
theorem
topology
time elapsed: 0.0148830413818


In [55]:
closest_word("fish",W,6)

('closest words to fish', 'are: ')
eat
eggs
fruit
meat
seeds
time elapsed: 0.0137209892273


# (e) Solve Analogies 

In [14]:
def solve_analogies(X,Y,Z,W,num=6):
    print("closest solution for analogy: ", X,":",Y,"::",Z,":","?")
    vec = W[vocab_index[Y]] - W[vocab_index[X]] + W[vocab_index[Z]]
    diff_W = W - np.array([vec]*W.shape[0])
    diff = [np.dot(x,x) for x in diff_W]
    close = np.argpartition(diff,num)[:num]
    
    for i in close:
        print(vocab[i])

In [57]:
X = 'france'
Y = 'paris'
Z = 'england'
solve_analogies(X,Y,Z,W)

('closest solution for analogy: ', 'france', ':', 'paris', '::', 'england', ':', '?')
england
london
oxford
dublin
cambridge
edinburgh


In [60]:
import pickle
with open("my_W_0","wb") as f:
    pickle.dump(W,f)

In [12]:
import pickle
with open("my_W_0","rb") as f:
    W = pickle.load(f)

In [15]:
X = 'france'
Y = 'paris'
Z = 'england'
solve_analogies(X,Y,Z,W)

('closest solution for analogy: ', 'france', ':', 'paris', '::', 'england', ':', '?')
england
london
oxford
dublin
cambridge
edinburgh


In [16]:
X = 'japan'
Y = 'tokyo'
Z = 'china'
solve_analogies(X,Y,Z,W)

('closest solution for analogy: ', 'japan', ':', 'tokyo', '::', 'china', ':', '?')
hong
shanghai
beijing
tokyo
taiwan
delhi


In [20]:
X = 'washington'
Y = 'america'
Z = 'mao'
solve_analogies(X,Y,Z,W)

('closest solution for analogy: ', 'washington', ':', 'america', '::', 'mao', ':', '?')
prevalent
peasant
domination
nationalism
malay
revolutions


In [21]:
X = 'capitalism'
Y = 'money'
Z = 'communism'
solve_analogies(X,Y,Z,W)

('closest solution for analogy: ', 'capitalism', ':', 'money', '::', 'communism', ':', '?')
money
pay
dollars
paid
cash
buy


In [24]:
X = 'girl'
Y = 'pretty'
Z = 'boy'
solve_analogies(X,Y,Z,W)

('closest solution for analogy: ', 'girl', ':', 'pretty', '::', 'boy', ':', '?')
bug
spell
thanks
pretty
pocket
unfortunately


In [25]:
X = 'soldier'
Y = 'army'
Z = 'student'
solve_analogies(X,Y,Z,W)

('closest solution for analogy: ', 'soldier', ':', 'army', '::', 'student', ':', '?')
education
students
student
schools
army
military
